In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers, optimizers
from keras.layers.normalization import BatchNormalization

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Analisando as especificações dos datasets de treinamento e teste, foi possível notar que eles contam com mais de 55 Milhões de Linhas. Com o intuito de reduzir o tamanho e melhorar o processamento do Kernel, vamos definir nosso conjunto de treinamento com menos linhas.

Além disso, vamos verificar quais são os tipos de features que encontramos nesse data set. 

In [ ]:
train_df =  pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv', nrows = 1_000_000)
train_df.dtypes

É interessante observar quatro campos especificos:

* pickup_longitude
* pickup_latitude
* dropoff_longitude
* dropoff_latitude

Com eles criaremos duas novas features traçarmos a rota da corrida do taxi.

In [ ]:
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_df)

Outro ponto que devemos tomar cuidado ao trabalhar com os datasets é sobre os dados nulos.

In [ ]:
print(train_df.isnull().sum())

Analisando nosso resultado, percebemos que em comparação com o todo, há apenas 69 registros nulos que representa uma porcentagem quase insignificante. Por isso, iremos apenas ignoraremos esses dados.

In [ ]:
print('Tamanho antes: %d' % len(train_df))
train_df = train_df.dropna(how = 'any', axis = 'rows')
print('Após exclusão: %d' % len(train_df))

Outro ponto interessante de se notar, é que podem existir casos em que as corridas não terminaram ou iniciaram em New York. E como podemos perceber isso? Bom, olhando para os graus, a diferença de 1 grau representa o percurso de 69 milhas. Por isso, ignoraremos dados em que a diferença seja maior que 5. 

Outro ponto que vamos ignorar no dataset são aqueles que começaram e terminaram no mesmo lugar.

Como nossa feature de distância já está criada, vamos agora dropar as colunas extras de latitude e longitude.



In [ ]:
print('Tamanho antes do tratamento: %d' % len(train_df))
train_df = train_df[(train_df.abs_diff_longitude < 5.0) & (train_df.abs_diff_latitude < 5.0)]
train_df = train_df[(train_df.abs_diff_longitude != 0) & (train_df.abs_diff_latitude != 0)]
dropped_columns = ['pickup_longitude', 'pickup_latitude', 
                   'dropoff_longitude', 'dropoff_latitude']
train = train_df.drop(dropped_columns, axis=1)
print('Tratamento após o tratamento: %d' % len(train))

Agora faremos um pouco de feature engineering e processamento dos dados.

Serão criadas três features:
    * Year, Month, Day, Hour, Weekday
    * Night (between 16h and 20h, from monday to friday)
    * Late night (between 20h and and 6h)

In [ ]:
def late_night (row):
    if (row['hour'] <= 6) or (row['hour'] >= 20):
        return 1
    else:
        return 0


def night (row):
    if ((row['hour'] <= 20) and (row['hour'] >= 16)) and (row['weekday'] < 5):
        return 1
    else:
        return 0
    
    
def manhattan(pickup_lat, pickup_long, dropoff_lat, dropoff_long):
    return np.abs(dropoff_lat - pickup_lat) + np.abs(dropoff_long - pickup_long)


def add_time_features(df):
    df['pickup_datetime'] =  pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
    df['year'] = df['pickup_datetime'].apply(lambda x: x.year)
    df['month'] = df['pickup_datetime'].apply(lambda x: x.month)
    df['day'] = df['pickup_datetime'].apply(lambda x: x.day)
    df['hour'] = df['pickup_datetime'].apply(lambda x: x.hour)
    df['weekday'] = df['pickup_datetime'].apply(lambda x: x.weekday())
    df['pickup_datetime'] =  df['pickup_datetime'].apply(lambda x: str(x))
    df['night'] = df.apply (lambda x: night(x), axis=1)
    df['late_night'] = df.apply (lambda x: late_night(x), axis=1)
    # Drop 'pickup_datetime' as we won't need it anymore
    df = df.drop('pickup_datetime', axis=1)
    
    return df



In [ ]:
train = add_time_features(train)

In [ ]:
train.head(5)

Para continuar o trabalho, vamos droppar duas colunas que não vão nos ajudar muito daqui pra frente: Key e Passenger_Count 

In [ ]:
dropped_columns2 = ['key', 'passenger_count']
train = train.drop(dropped_columns2, axis=1)
train.head(5)

Vamos Splitar o dataset de treino na proporção Treino 90% e Validação 10%.

In [ ]:
train_df, validation_df = train_test_split(train, test_size=0.10, random_state=1)

# Get labels
train_labels = train_df['fare_amount'].values
validation_labels = validation_df['fare_amount'].values
train_df = train_df.drop(['fare_amount'], axis=1)
validation_df = validation_df.drop(['fare_amount'], axis=1)

In [ ]:
print(len(train_df))
print(len(validation_df))

In [ ]:
scaler = preprocessing.MinMaxScaler()
train_df_scaled = scaler.fit_transform(train_df)
validation_df_scaled = scaler.transform(validation_df)


Com os nossos datasets de treino e validação prontos, partiremos para preparar o nosso dataset de test

In [ ]:
test = pd.read_csv('../input/new-york-city-taxi-fare-prediction/test.csv')
print(len(test))
#vamos limpar um pouco do dataset
add_travel_vector_features(test)
test = test.drop(dropped_columns, axis=1)

test_aux = test.drop(['passenger_count'], axis=1)
test = test.drop(dropped_columns2, axis=1)

# vamos agora adicionar a feature criada para datas
test = add_time_features(test)

test_df_scaled = scaler.transform(test)

test.head(5)

O nosso modelo sera sequencial e os parametros serão:

* Função de Ativação: ReLu
* Otimização: Adam
* Função de Perda: MSE
* Epocas: 20
* Learnin Rate: 0.1
* Batch Size: 256


In [ ]:
# Model parameters
BATCH_SIZE = 512
EPOCHS = 20
LEARNING_RATE = 0.001
DATASET_SIZE = 1000000

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=train_df_scaled.shape[1], activity_regularizer=regularizers.l1(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))

sgd = optimizers.SGD(lr=LEARNING_RATE, clipvalue=0.5)
#adam = optimizers.adam(lr=LEARNING_RATE)
model.compile(loss='mse', optimizer=sgd, metrics=['mae'])

In [ ]:
history = model.fit(x=train_df_scaled, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                    verbose=1, validation_data=(validation_df_scaled, validation_labels), 
                    shuffle=True)

In [ ]:
prediction = model.predict(test_df_scaled, batch_size=128, verbose=1)

print(prediction)
